##### import libraries

In [1]:
import json 
import cv2
from matplotlib import pyplot as plt
import numpy as np
from preprocess import *
import SimpleITK as sitk
from collections import OrderedDict
import os
from register_images import *
import sys
sys.path.insert(0, '../')
from parse_registration_json import ParserRegistrationJson
from parse_study_dict import ParserStudyDict
import time

In [2]:
####### INPUTS
json_path = "./jsonData/TCIA_FUSION.json"
preprocess_moving = True
preprocess_fixed = True
run_registration = True
extension = 'nii.gz'
timings = {}

try:
    with open('coord.txt') as f:
        coord = json.load(f)   
        
except:
    coord = {}

############### START REGISTRATION HERE

json_obj = ParserRegistrationJson(json_path)

studies = json_obj.studies
toProcess = json_obj.ToProcess
outputPath = json_obj.output_path

cases = toProcess.keys()

if not os.path.isdir(outputPath):
    os.mkdir(outputPath) 

###### PREPROCESSING DESTINATIONS ######################################
preprocess_moving_dest = outputPath + '/preprocess/hist'
preprocess_fixed_dest = outputPath + '/preprocess/mri'

if not os.path.isdir(outputPath + '/preprocess'):
    os.mkdir(outputPath + '/preprocess')

if not os.path.isdir(preprocess_moving_dest):
    os.mkdir(preprocess_moving_dest)
    
if not os.path.isdir(preprocess_fixed_dest):
    os.mkdir(preprocess_fixed_dest)

'do_affine'
'do_deformable'
'do_reconstruction'
'fast_execution'
'use_imaging_constraints'
Reading aaa0069 Study Json ./jsonData/reg_aaa0069.json


In [4]:
# start doing preprocessing on each case and register
for s in json_obj.studies:
    if json_obj.ToProcess:
        if not (s in json_obj.ToProcess):
            print("Skipping", s)
            continue

    print("x"*30, "Processing", s,"x"*30)
    studyDict = json_obj.studies[s] 


    studyParser = ParserStudyDict(studyDict)
    
    sid = studyParser.id
    fixed_img_mha = studyParser.fixed_filename
    fixed_seg = studyParser.fixed_segmentation_filename
    moving_dict = studyParser.ReadMovingImage()
    
    ###### PREPROCESSING HISTOLOGY HERE #############################################################
    if preprocess_moving == True: 
        print('Preprocessing moving sid:', sid, '...')
        preprocess_hist(moving_dict, preprocess_moving_dest, sid)
        print('Finished preprocessing', sid)

    ###### PREPROCESSING MRI HERE #############################################################
    if preprocess_fixed == True:
        print ("Preprocessing fixed case:", sid, '...')

        coord = preprocess_mri(fixed_img_mha, fixed_seg, preprocess_fixed_dest, coord, sid)

        print("Finished processing fixed mha", sid)

        with open('coord.txt', 'w') as json_file: 
            json.dump(coord, json_file)
    ##### ALIGNMENT HERE ########################################################################
    if run_registration == True: 
        
        ######## LOAD MODELS
        print('.'*30, 'Begin deep learning registration for ' + sid + '.'*30)

        try:
            model_cache
        except NameError:
            feature_extraction_cnn = 'resnet101'

            model_aff_path = './trained_models/best_CombinedLoss_affine_resnet101.pth.tar'
            model_tps_path = './trained_models/best_CombinedLoss_tps_resnet101.pth.tar'

            model_cache = load_models(feature_extraction_cnn, model_aff_path, model_tps_path, do_deformable=True)
        
        start = time.time()
        output3D_cache = register(preprocess_moving_dest + sid + '/', preprocess_fixed_dest + sid + '/', coord, model_cache, sid)
        out3Dhist_highRes, out3Dmri_highRes, out3Dcancer_highRes, out3D_region00, out3D_region10, out3D_region09, out3Dmri_mask = output3D_cache
        end = time.time()
        print("Registration done in {:6.3f}(min)".format((end-start)/60.0))
        imMri = sitk.ReadImage(fixed_img_mha)
        mriOrigin = imMri[:,:,coord[sid]['slice'][0]:coord[sid]['slice'][-1]].GetOrigin()
        mriSpace  = imMri.GetSpacing()
        mriDirection = imMri.GetDirection()

        imSpatialInfo = (mriOrigin, mriSpace, mriDirection)

        # write output hist 3D volume to .nii.gz format
        fn_moving_highRes = '_moved_highres_rgb.'
        output_results_high_res(preprocess_moving_dest + sid + '/',preprocess_fixed_dest,outputPath, out3Dhist_highRes, sid, fn_moving_highRes, imSpatialInfo, coord, imMri, extension = "nii.gz")

        #write output mri 3D volume to .nii.gz format
        fn_fixed_highRes = '_fixed_image.'
        output_results(outputPath, out3Dmri_highRes, sid, fn_fixed_highRes, imSpatialInfo, extension = "nii.gz")

        #write output cancer outline 3D volume to .nii.gz format
        fn_cancer_highRes = '_moved_highres_region01_label.'
        output_results_high_res(preprocess_moving_dest + sid + '/',preprocess_fixed_dest,outputPath, out3Dcancer_highRes, sid, fn_cancer_highRes, imSpatialInfo, coord, imMri, extension = "nii.gz")
        
        #write region00
        fn_region00 = '_moved_highres_region00_label.'
        output_results_high_res(preprocess_moving_dest + sid + '/',preprocess_fixed_dest,outputPath, out3D_region00, sid, fn_region00, imSpatialInfo, coord, imMri, extension = "nii.gz")
        
        #write region10
        fn_region00 = '_moved_highres_region10_label.'
        output_results_high_res(preprocess_moving_dest + sid + '/',preprocess_fixed_dest,outputPath, out3D_region10, sid, fn_region00, imSpatialInfo, coord, imMri, extension = "nii.gz")
        
        #write region09
        fn_region00 = '_moved_highres_region09_label.'
        output_results_high_res(preprocess_moving_dest + sid + '/',preprocess_fixed_dest,outputPath, out3D_region09, sid, fn_region00, imSpatialInfo, coord, imMri, extension = "nii.gz")
        
        #write mriMask
        fn_mriMask = '_fixed_mask_label.'
        output_results(outputPath, out3Dmri_mask, sid, fn_mriMask, imSpatialInfo, extension = "nii.gz")

        timings[s] = (end-start)/60.0
        print('Done!')

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx Processing aaa0069 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Preprocessing moving sid: aaa0069 ...
Finished preprocessing aaa0069
Preprocessing fixed case: aaa0069 ...
Finished processing fixed mha aaa0069
.............................. Begin deep learning registration for aaa0069..............................


AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [ ]:
json.dump(timings, open("timings.txt",'w'))